# APPENDIX C: TABLE INDICATING THE SUCCESS RATE

In [1]:
import os
import sys
import glob
from scipy import stats
sys.path.insert(0, '/cs/sci/freundlich/CUSPCORE/Analysis/Model')
import cuspcore_go
reload(cuspcore_go)
from cuspcore_aux import *
import pickle
import numpy
import scipy
from scipy.special import erf

sys.path.insert(0, '/cs/sci/freundlich/CUSPCORE/')

Ttype='Tmulti'
rmin_evolve=10**-1.75
rmax_evolve=1.
linear_slopes=False
t_min=3.
merger_thr=1.

sims=[sim[52:60] for sim in glob.glob('/cs/sci/freundlich/CUSPCORE/ARTICLE/output_20190503_g[!s]*[!temp].pickle')]
print 'Number of galaxies:', size(sims)

t1_array_all=[]
t2_array_all=[]
Dsnapshot_all=[]
delta_all=[]
delta_i_all=[]
fmean_all=[]
fstd_all=[]
fmerger_all=[]
fprofile_all=[]
fprofile_simname=[]

for sim in sims:
    output_file_sim='/cs/sci/freundlich/CUSPCORE/ARTICLE/output_20190503_%s.pickle'%sim
    with open(output_file_sim) as f:
        output_sim = pickle.load(f)
    
    [percent_tot,t1_array,t2_array,Dsnapshot,delta,delta_i,fmean,fstd,fmerger,fprofile, fprofile_simname,criteria_threshold]=output_sim
    t1_array_all.append(t1_array[0])
    t2_array_all.append(t2_array[0])
    Dsnapshot_all.append(Dsnapshot[0])
    delta_all.append(delta[0])
    delta_i_all.append(delta_i[0])
    fmean_all.append(fmean[0])
    fstd_all.append(fstd[0])
    fmerger_all.append(fmerger[0])
    fprofile_all.append(fprofile)
    
output=[percent_tot,t1_array_all,t2_array_all,Dsnapshot_all,delta_all,delta_i_all,fmean_all,fstd_all,fmerger_all,fprofile_all, fprofile_simname,criteria_threshold]
count_output=count_successes_all(sims,output,merger_threshold=0.1,fit_threshold=0.05)

print 'Load: Done!'

########################################################

[percent_tot,t1_array_all,t2_array_all,Dsnapshot_all,delta_all,delta_i_all,fmean_all,fstd_all,fmerger_all,fprofile_all, fprofile_simname,criteria_threshold] =output
t_min,Dsnapshot_threshold,merger_thr,Dfit_threshold,fmean_min = criteria_threshold
ngalaxies = shape(t1_array_all)[0]

t_min=4

########################################################

##### LOAD PROPERTY NAMES
catalog='/cs/sci/freundlich/CUSPCORE/catalogs/NIHAO_a%.4f.txt'%1.
colnames=genfromtxt(catalog,max_rows=1,dtype='S20')

##### LOAD TIMES
import pickle
directory='/cs/sci/freundlich/CUSPCORE/NIHAO_data/'
with open(directory+'NIHAO-%s.pickle'%sims[0][1:]) as f:
            gl = pickle.load(f)

a_array=[]
for (i,ss) in zip(range(size(gl)),gl):
            a_array.append(ss['a'])         
        
a_array=array(a_array)
z_array = 1./a_array - 1.
logz=log10(z_array)

time=array(t2_array_all).flatten().copy()

n_outputs=size(a_array)
sim_list=[]
i_array=[]
for i in range(size(sims)):
    sim_list=sim_list+[sims[i]]*n_outputs
    i_array=i_array+[i for i in range(n_outputs)]

sim_list=array(sim_list)
i_array=array(i_array)

#### GET DELTA AND SUCCESS 
delta=array(delta_all).flatten().copy()

dmax=0.4
delta_red=delta.copy()
delta_red[time<t_min]=nan*ones_like(delta_red[time<t_min])

dsim=abs(array(Dsnapshot_all).flatten().copy())
dsim_red=dsim.copy()

#### MASS RATIOS
frms=array(fmean_all).flatten().copy()
frms[frms==-inf]=nan
fmerger=abs(array(fmerger_all).flatten()).copy()
fmerger[fmerger==-inf]=nan
previous_fmerger=array([nan]+fmerger[:-1].tolist()).copy()

fmax=0.35
frms_red=frms.copy()
fmerger_red=fmerger.copy()
previous_fmerger_red=previous_fmerger.copy()

frms_red[log10(frms_red)==-inf]=nan
fmerger_red[log10(fmerger_red)==-inf]=nan

frms_red[log10(frms_red)<-2.5]=nan*ones_like(frms_red[log10(frms_red)<-2.5])
fmerger_red[log10(fmerger_red)<-2.5]=nan*ones_like(fmerger_red[log10(fmerger_red)<-2.5])
frms_red[log10(frms_red)>1]=nan*ones_like(frms_red[log10(frms_red)>1])
fmerger_red[log10(fmerger_red)>1]=nan*ones_like(fmerger_red[log10(fmerger_red)>1])


#### QUANTITIES FROM FANGZHOU JIANG'S CATALOG

mvir=[]
mstar=[]
mgas=[]
mcold=[]
SFR=[]
re_star=[]
n_sersic=[]
cnfw=[]
q=[]
p=[]        
    
for ID in sims:
    for a in a_array:
        if a==a_array[0] or a ==a_array[1]:
            mvir.append(nan)
            mstar.append(nan)
            mgas.append(nan)
            mcold.append(nan)
            SFR.append(nan)
            re_star.append(nan)
            n_sersic.append(nan)
            cnfw.append(nan)
            p.append(nan)
            q.append(nan)
        else:
            catalog='/cs/sci/freundlich/CUSPCORE/catalogs/NIHAO_a%.4f.txt'%a
            data=genfromtxt(catalog,skip_header =1)
            try:
                i_ID=where(data[:,0]==float(ID[1:]))[0][0]
                mvir.append(data[i_ID,7])
                mstar.append(data[i_ID,17])
                mgas.append(data[i_ID,18])
                mcold.append(data[i_ID,19])
                SFR.append(data[i_ID,20])
                re_star.append(data[i_ID,21])
                n_sersic.append(data[i_ID,32])
                cnfw.append(data[i_ID,12])
                q.append(data[i_ID,33])
                p.append(data[i_ID,34])    
            except:
                mvir.append(nan)
                mstar.append(nan)
                mgas.append(nan)
                mcold.append(nan)
                SFR.append(nan)
                re_star.append(nan)
                n_sersic.append(nan)
                cnfw.append(nan)
                q.append(nan)
                p.append(nan)

mvir=array(mvir)
mstar=array(mstar)
mgas=array(mgas)
mcold=array(mcold)
SFR=array(SFR)
re_star=array(re_star)
n_sersic=array(n_sersic)
cnfw=array(cnfw)
q=array(q)
p=array(p)

f=sqrt(1-p**2)
e=sqrt(1-q**2)
ef=e-f
epf=e+f

mugas=mgas/mstar
muhot=(mgas-mcold)/mstar
mucold=mcold/mstar

mugas_red=mugas.copy()
mugas_red[mugas_red>50]=nan*ones_like(mugas_red[mugas_red>50])
mucold_red=mucold.copy()
mucold_red[mucold_red>50]=nan*ones_like(mucold_red[mucold_red>50])

fgas=mgas/(mstar+mgas+mvir)
fcold=mcold/(mstar+mgas)
fhot=(mgas-mcold)/(mstar+mgas)
fb=(mgas+mstar)/(mgas+mstar+mvir)
fb_gas=mgas/(mgas+mstar+mvir)
fb_star=mstar/(mgas+mstar+mvir)

print 'Done!'

/cs/sci/sharonl/miniconda/envs/iyt32/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Number of galaxies: 33
g1.57e11: 22/50 = 44 percent success
        : 15/30 = 50 percent success without mergers
        : 12/23 = 52 percent success when fmean > 0.06
        : 3/7 = 43 percent success when fmean < 0.06
g4.99e10: 27/50 = 54 percent success
        : 27/41 = 66 percent success without mergers
        : 12/23 = 52 percent success when fmean > 0.06
        : 15/18 = 83 percent success when fmean < 0.06
g3.21e11: 18/50 = 36 percent success
        : 16/33 = 48 percent success without mergers
        : 11/26 = 42 percent success when fmean > 0.06
        : 5/7 = 71 percent success when fmean < 0.06
g3.44e10: 18/50 = 36 percent success
        : 17/24 = 71 percent success without mergers
        : 13/16 = 81 percent success when fmean > 0.06
        : 4/8 = 50 percent success when fmean < 0.06
g3.59e11: 19/50 = 38 percent success
        : 17/25 = 68 percent success without mergers
        : 12/17 = 71 percent success when fmean > 0.06
        : 5/8 = 62 percent success whe

In [4]:
import cuspcore_aux
from cuspcore_aux import *

[percent_all_tot,percent_nomergers_tot,percent_fmean_tot,percent_others_tot]=successes_table_all(sims,output,merger_threshold=0.1,fit_threshold=0.1,fmean_threshold=0.07,t_min=3,use_previous_mergers=0,use_cond2=True)
print 'Done!'

Number of galaxies:  33
texttt{g1.57e11}& $1.78 \times 10^{11}$& $148.6$& $1.15 \times 10^{9}$& $5.42$ & $23.62$& 29/50 = 58\percent& 24/36 = 67\percent& 18/23 = 78\percent& 06/13 = 46\percent endline
texttt{g4.99e10}$^\dagger$& $5.71 \times 10^{10}$& $101.8$& $1.22 \times 10^{8}$& $3.01$ & $24.16$& 31/43 = 72\percent& 31/42 = 74\percent& 13/23 = 57\percent& 18/19 = 95\percent endline
texttt{g3.21e11}& $3.95 \times 10^{11}$& $193.9$& $3.67 \times 10^{9}$& $4.88$ & $21.29$& 25/48 = 52\percent& 25/39 = 64\percent& 18/29 = 62\percent& 07/10 = 70\percent endline
texttt{g3.44e10}$^\dagger$& $6.65 \times 10^{10}$& $107.0$& $6.32 \times 10^{7}$& $2.54$ & $24.43$& 21/33 = 64\percent& 19/26 = 73\percent& 13/15 = 87\percent& 06/11 = 55\percent endline
texttt{g3.59e11}& $4.44 \times 10^{11}$& $201.6$& $4.36 \times 10^{9}$& $5.10$ & $21.76$& 26/46 = 57\percent& 24/33 = 73\percent& 14/19 = 74\percent& 10/14 = 71\percent endline
texttt{g6.12e10}$^\dagger$& $5.5 \times 10^{10}$& $100.5$& $9.13 \times